# Donkey Colab with GPU

Donkey Colab with GPU.

In [0]:
!git clone -b dev https://github.com/autorope/donkeycar.git donkey

Cloning into 'donkey'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 6688 (delta 0), reused 0 (delta 0), pack-reused 6687
Receiving objects: 100% (6688/6688), 41.53 MiB | 38.17 MiB/s, done.
Resolving deltas: 100% (3853/3853), done.


In [0]:
!pip3 install -e donkey

Obtaining file:///content/donkey
  Found existing installation: donkeycar 2.5.7
    Can't uninstall 'donkeycar'. No files were found to uninstall.
  Running setup.py develop for donkeycar


In [0]:
!donkey createcar --template donkey2  /content/mycar

using donkey version: 2.5.7 ...
Creating car folder: /content/mycar
making dir  /content/mycar
Creating data & model folders.
making dir  /content/mycar/models
making dir  /content/mycar/data
making dir  /content/mycar/logs
Car app already exists. Delete it and rerun createcar to replace.
Car config already exists. Delete it and rerun createcar to replace.
Donkey setup complete.


In [39]:
!mkdir /content/mycar/data/tub001

!mv datas.zip /content/mycar/data/tub001/

mkdir: cannot create directory ‘/content/mycar/data/tub001’: File exists
mv: 'datas.zip' and '/content/mycar/data/tub001/datas.zip' are the same file
/content/tub001
unzip:  cannot find or open datas001.zip, datas001.zip.zip or datas001.zip.ZIP.


In [45]:
%cd /content/mycar/data/tub001

!unzip datas001.zip

/content/mycar/data/tub001
unzip:  cannot find or open datas001.zip, datas001.zip.zip or datas001.zip.ZIP.


In [46]:
#!/usr/bin/env python3

import os

from docopt import docopt
import donkeycar as dk
import tensorflow as tf
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.actuator import PCA9685, PWMSteering, PWMThrottle
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.web_controller import LocalWebController
from donkeycar.parts.clock import Timestamp
from donkeycar.parts.datastore import TubGroup, TubWriter
from donkeycar.parts.keras import KerasLinear
from donkeycar.parts.transform import Lambda


def train(cfg, tub_names, new_model_path, base_model_path=None):
    """
    use the specified data in tub_names to train an artifical neural network
    saves the output trained model as model_name
    """
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    new_model_path = os.path.expanduser(new_model_path)

    kl = KerasLinear()
    if base_model_path is not None:
        base_model_path = os.path.expanduser(base_model_path)
        kl.load(base_model_path)

    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join("/content/mycar/data/", '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys,
                                                    batch_size=128,
                                                    train_frac=0.8)

    total_records = len(tubgroup.df)
    total_train = int(total_records * 0.8)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train //128
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=new_model_path,
             steps=steps_per_epoch,
             train_split=0.8)


if __name__ == '__main__':
    with tf.device('/gpu:0'):
      cfg = None

      tub = None
      new_model_path = "/content/mycar/models/mypilot"
      base_model_path = None
      cache = None
      train(cfg, tub, new_model_path, base_model_path)

tub_names None
train: 1626, validation: 407
steps_per_epoch 12
Epoch 1/100
11/12 [==========================>...] - ETA: 0s - loss: 72.7476 - angle_out_loss: 127.1892 - throttle_out_loss: 18.3060
Epoch 00001: val_loss improved from inf to 0.48132, saving model to /content/mycar/models/mypilot
12/12 [==============================] - 8s 680ms/step - loss: 66.7440 - angle_out_loss: 116.6473 - throttle_out_loss: 16.8408 - val_loss: 0.4813 - val_angle_out_loss: 0.7452 - val_throttle_out_loss: 0.2174
Epoch 2/100
11/12 [==========================>...] - ETA: 0s - loss: 0.2371 - angle_out_loss: 0.2787 - throttle_out_loss: 0.1955
Epoch 00002: val_loss improved from 0.48132 to 0.10142, saving model to /content/mycar/models/mypilot
12/12 [==============================] - 3s 212ms/step - loss: 0.2295 - angle_out_loss: 0.2721 - throttle_out_loss: 0.1869 - val_loss: 0.1014 - val_angle_out_loss: 0.1002 - val_throttle_out_loss: 0.1026
Epoch 3/100
11/12 [==========================>...] - ETA: 0s - lo

KeyboardInterrupt: ignored